In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


df_pivot = pd.read_csv('pivot_table.csv', index_col=0)
df_melt = df_pivot.stack().reset_index().rename(columns={'level_1': 'Movie_Id', 0: 'Rating'})


user_movie_matrix = df_pivot.values
n_users, n_movies = user_movie_matrix.shape


scaler = StandardScaler()
user_movie_matrix_normalized = scaler.fit_transform(user_movie_matrix)


def flatten_matrix(matrix):
    return [(i, j, matrix[i, j]) for i in range(matrix.shape[0]) for j in range(matrix.shape[1]) if matrix[i, j] > 0]

ratings_flat = flatten_matrix(user_movie_matrix_normalized)
train_data, test_data = train_test_split(ratings_flat, test_size=0.2, random_state=42)


def to_tensor(data, n_users, n_movies):
    matrix = torch.zeros(n_users, n_movies)
    for i, j, r in data:
        matrix[int(i), int(j)] = r
    return matrix

train_tensor = to_tensor(train_data, n_users, n_movies)
test_tensor = to_tensor(test_data, n_users, n_movies)


class RBM(nn.Module):
    def __init__(self, n_visible, n_hidden):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_visible, n_hidden) * 0.1)
        self.b = nn.Parameter(torch.zeros(n_visible))
        self.c = nn.Parameter(torch.zeros(n_hidden))
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, v):
        h_prob = torch.relu(torch.matmul(v, self.W) + self.c)  
        h_prob = self.dropout(h_prob)
        v_recon = torch.sigmoid(torch.matmul(h_prob, self.W.t()) + self.b)
        return v_recon


learning_rates = [0.5,0.005, 0.01]
hidden_layers = [100, 200, 300]
weight_decays = [0.0001, 0.001]
results = []

n
for lr in learning_rates:
    for n_hidden in hidden_layers:
        for wd in weight_decays:
            print(f'Training RBM with LR={lr}, Hidden Layers={n_hidden}, Weight Decay={wd}')
            
            rbm = RBM(n_visible=n_movies, n_hidden=n_hidden)
            optimizer = optim.Adam(rbm.parameters(), lr=lr, weight_decay=wd)
            
            
            def train_rbm(model, train_data, optimizer, epochs=100):  
                for epoch in range(epochs):
                    model.train()
                    optimizer.zero_grad()
                    recon = model(train_data)
                    loss = nn.MSELoss()(recon, train_data)
                    loss.backward()
                    optimizer.step()
                    if epoch % 10 == 0:
                        print(f'Epoch {epoch}: Loss={loss.item()}')
            
            train_rbm(rbm, train_tensor, optimizer)
            
            
            def evaluate_rbm(model, test_data):
                model.eval()
                with torch.no_grad():
                    pred = model(test_tensor)
                    pred = pred.numpy()
                    test_matrix = test_tensor.numpy()

                    
                    test_flat = [(i, j, test_matrix[i, j]) for i in range(test_matrix.shape[0]) for j in range(test_matrix.shape[1]) if test_matrix[i, j] > 0]
                    pred_flat = [(i, j, pred[i, j]) for i in range(pred.shape[0]) for j in range(pred.shape[1]) if test_matrix[i, j] > 0]

                    
                    test_ratings = np.array([r for _, _, r in test_flat])
                    pred_ratings = np.array([r for _, _, r in pred_flat])

                    rmse = np.sqrt(mean_squared_error(test_ratings, pred_ratings))
                    mae = mean_absolute_error(test_ratings, pred_ratings)
                    return rmse, mae

            rmse, mae = evaluate_rbm(rbm, test_tensor)
            results.append((lr, n_hidden, wd, rmse, mae))
            print(f'LR={lr}, Hidden Layers={n_hidden}, Weight Decay={wd} => RMSE: {rmse}, MAE: {mae}')


results_df = pd.DataFrame(results, columns=['Learning Rate', 'Hidden Layers', 'Weight Decay', 'RMSE', 'MAE'])
print(results_df)




Training RBM with LR=0.5, Hidden Layers=100, Weight Decay=0.0001
Epoch 0: Loss=0.2633781433105469
Epoch 10: Loss=0.3737131357192993
Epoch 20: Loss=0.12262508273124695
Epoch 30: Loss=0.10167749226093292
Epoch 40: Loss=0.08068560063838959
Epoch 50: Loss=0.08107228577136993
Epoch 60: Loss=0.08172430843114853
Epoch 70: Loss=0.07595827430486679
Epoch 80: Loss=0.07606039196252823
Epoch 90: Loss=0.07731418311595917
LR=0.5, Hidden Layers=100, Weight Decay=0.0001 => RMSE: 0.7324761748313904, MAE: 0.5883461236953735
Training RBM with LR=0.5, Hidden Layers=100, Weight Decay=0.001
Epoch 0: Loss=0.2627525329589844
Epoch 10: Loss=0.19310899078845978
Epoch 20: Loss=0.15837927162647247
Epoch 30: Loss=0.0867263525724411
Epoch 40: Loss=0.0840994268655777
Epoch 50: Loss=0.08360791206359863
Epoch 60: Loss=0.08656520396471024
Epoch 70: Loss=0.08499704301357269
Epoch 80: Loss=0.0967404767870903
Epoch 90: Loss=0.09367476403713226
LR=0.5, Hidden Layers=100, Weight Decay=0.001 => RMSE: 0.6598947048187256, MAE: